In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent
sys.path.append(str(BASE_DIR))

In [10]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import pandas as pd

In [11]:
DIR = BASE_DIR/'scripts_res'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res')

### ASBO

In [12]:
re_run = True
re_run_save = True

In [ ]:
sol = 'ASBO'
case_name = 'Case2'
runs_number = 100

obj_list = []
x_1_list = []
x_2_list = []
con_1_list  = []

def func(x1,x2):
    x = np.array([x1,x2])
    obj, c1, = Case2(x)
    cons = [c1]
    return -obj,c1

nc_lb = np.array([-1e6])
nc_ub = np.array([0,])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 10),'x2': (0, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

if re_run == True or re_run_save == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=100, n_iter=50,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        con_1_list.extend([optimizer.max['constraint']])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


In [14]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1
count,5.000000,5.000000,5.000000,5.000000
mean,268.645481,3.254099,0.062189,-0.018017
std,0.007572,0.010842,0.005073,0.001125
min,268.633884,3.241863,0.054593,-0.019804
25%,268.643058,3.247534,0.061334,-0.018406
50%,268.647492,3.255067,0.061681,-0.017529
75%,268.648948,3.255352,0.065233,-0.017351
max,268.654021,3.270679,0.068106,-0.016998


### trust-constr

In [33]:
re_run = True
re_run_save = True

In [34]:
sol = 'trust-constr'
list_realvalue = []
case_name = 'Case2'
runs_number = 100

initial_x1 = []
initial_x2 = []
x_1_list = []
x_2_list = []
obj_list = []
con_1_list  = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1= Case2(x)
    cons = [c1]
    list_realvalue.extend([[obj]+cons])
    return obj,c1

def recyc(x):
    nc_lb = np.array([-1e6])
    nc_ub = np.array([0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-5,0] 
    ub = [10,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
    con_1_list.append(result.constr[0])
    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run == True or re_run_save == True:
    x_random = initial_random(lb=[-5,0],ub=[10,15])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


obj =  -268.78838103316497 , x =  [3.27118983 0.04969917]
obj =  -200.66282779946073 , x =  [-4.16412556 14.79906521]
obj =  -268.7883810416733 , x =  [3.27118983 0.04969918]
obj =  -200.66282780022857 , x =  [-4.16412556 14.79906523]
obj =  -268.7883808282727 , x =  [3.27118982 0.04969919]


In [35]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,initial_x1,initial_x2,nfev
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,-241.538160,0.297064,5.949446,0.730985,5.033045,122.400000
std,37.313902,4.072490,8.078560,1.994668,3.278358,88.852687
min,-268.788381,-4.164126,0.049699,-1.930270,0.388893,48.000000
25%,-268.788381,-4.164126,0.049699,-0.505180,4.002409,54.000000
50%,-268.788381,3.271190,0.049699,1.305517,4.955022,72.000000
75%,-200.662828,3.271190,14.799065,1.539924,6.529836,210.000000
max,-200.662828,3.271190,14.799065,3.244937,9.289064,228.000000


### slsqp

In [36]:
re_run = True
re_run_save = True

In [37]:
sol = 'slsqp'
list_realvalue = []
case_name = 'Case2'
runs_number = 100

initial_x1 = []
initial_x2 = []
x_1_list = []
x_2_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1= Case2(x)
    cons = [c1]
    list_realvalue.extend([[obj]+cons])
    return obj,c1

def recyc(x):
    nc_lb = np.array([-1e6])
    nc_ub = np.array([0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-5,0] 
    ub = [10,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
 
    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run == True or re_run_save == True:
    x_random = initial_random(lb=[-5,0],ub=[10,15])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)

obj =  -268.7885046713122 , x =  [3.27302382 0.04886974]
obj =  -268.78850467678114 , x =  [3.27302381 0.04886974]
obj =  -268.7885048274905 , x =  [3.27302386 0.04886971]
obj =  -268.7885054032581 , x =  [3.27302381 0.04886972]
obj =  -268.78850480881397 , x =  [3.27302374 0.04886977]


In [38]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,initial_x1,initial_x2,nfev
count,5.000000e+00,5.000000e+00,5.000000e+00,5.000000,5.000000,5.000000
mean,-2.687885e+02,3.273024e+00,4.886973e-02,0.730985,5.033045,42.000000
std,3.026722e-07,4.305535e-08,2.197878e-08,1.994668,3.278358,9.643651
min,-2.687885e+02,3.273024e+00,4.886971e-02,-1.930270,0.388893,30.000000
25%,-2.687885e+02,3.273024e+00,4.886972e-02,-0.505180,4.002409,33.000000
50%,-2.687885e+02,3.273024e+00,4.886974e-02,1.305517,4.955022,49.000000
75%,-2.687885e+02,3.273024e+00,4.886974e-02,1.539924,6.529836,49.000000
max,-2.687885e+02,3.273024e+00,4.886977e-02,3.244937,9.289064,49.000000


### CBO

In [39]:
re_run = True
re_run_save = True

In [40]:
sol = 'CBO'
case_name = 'Case2'
runs_number = 100

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

obj_list = []
x_1_list = []
x_2_list = []
con_1_list  = []

def func_obj(x1,x2):
    x = np.array([x1,x2])
    obj, c1 = Case2(x)
    cons = [c1]
    return -obj

def func_cons(x1,x2):
    x = np.array([x1,x2])
    obj, c1 = Case2(x)
    return c1

nc_lb = np.array([-1e6])
nc_ub = np.array([0,])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-5, 10),'x2': (0, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

if re_run == True or re_run_save == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=100, n_iter=50,acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        con_1_list.extend([optimizer.max['constraint']])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


{'target': 254.51117417656474, 'params': {'x1': 3.797694022151248, 'x2': 0.3016131928124033}, 'constraint': -0.3061983505082946}
{'target': 263.0013363474833, 'params': {'x1': 3.3957547308121363, 'x2': 0.1883397023867378}, 'constraint': -0.6958078432812682}
{'target': 252.83314556821014, 'params': {'x1': 3.740273327935519, 'x2': 0.3832641277453913}, 'constraint': -0.7678254093955195}
{'target': 240.59523336815826, 'params': {'x1': 2.710525897261629, 'x2': 1.30844053171207}, 'constraint': -1.9634887955332916}
{'target': 230.4210693623198, 'params': {'x1': 2.8254418366743703, 'x2': 1.6229007433084601}, 'constraint': -3.2952743975783445}


In [41]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1
count,5.000000,5.000000,5.000000,5.000000
mean,248.272392,3.293938,0.760912,-1.405719
std,12.789382,0.505769,0.656549,1.224883
min,230.421069,2.710526,0.188340,-3.295274
25%,240.595233,2.825442,0.301613,-1.963489
50%,252.833146,3.395755,0.383264,-0.767825
75%,254.511174,3.740273,1.308441,-0.695808
max,263.001336,3.797694,1.622901,-0.306198


### TuRBO

In [42]:
re_run = True
re_run_save = True

In [43]:
sol = 'TuRBO'
case_name = 'Case2'

temp_y = []
temp_x = []
x_1_list = []
x_2_list = []
iterations = []
max_values = []

runs_number = 100
itera = 0
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

def func(x):
    global itera
    obj, c1 = Case2(x)
    cons = [c1]
    itera=itera+1
    return -obj,c1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
dim = 2
batch_size = 1
n_init = 100 
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1e6, -1e6, -1e6, ], dtype=torch.float64)
ub =  torch.tensor([0, 0,0 ], dtype=torch.float64)
bounds = torch.tensor([[-5, 0,], [10, 15]])

def objective(x):
    res = func(x)
    target = res[0]
    c1 = res[1]
    return target,c1
def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []

            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1,= combined_tensor[0], combined_tensor[1]
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)

            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)

            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)

                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next,  = combined_tensor[0], combined_tensor[1]
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)

                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                bool_tensor = (C1 >= lb.repeat(len(C1), 1)) & (C1 <= ub.repeat(len(C1), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                Valid_X = train_X[bool_tensor]
                if len(Valid_Y)==0:
                    break
                else:
                    max_index = torch.argmax(Valid_Y)  
                    max_y = Valid_Y[max_index]         
                    corresponding_x = Valid_X[max_index]
                    temp_y.append(max_y)
                    temp_x.append(corresponding_x)
                    # list_realvalue.append(max(Valid_Y[0]))   # Wrong version
                if len(temp_y) == 50:
                    max_value = max(temp_y).item()
                    temp_y_np = np.array([t.item() for t in temp_y])
                    max_index = np.argmax(temp_y_np)
                    # max_index = np.argmax(temp_y)
                    c_x = temp_x[max_index]
                    print("obj= ",max_value, ' x = ', c_x)
                    max_values.append(max_value)
                    x_1_list.append(c_x[0].item())
                    x_2_list.append(c_x[1].item())
                    if len(iterations)==0:
                        iterations.append(itera)
                    else:
                        iterations.append(itera-sum(iterations))
                    temp_y.clear()
                    temp_x.clear()

                    df = pd.DataFrame({
                    'obj': max_values,
                    'x_1': x_1_list,
                    'x_2': x_2_list,
                    'iterations': iterations
                    })
                    if re_run_save == True:
                        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)                    
                    break
        except Exception as e:
            print(e)
            if len(temp_y)==0:
                print("Early stop and no feasible solutions")
                max_values.append(None)
                x_1_list.append(None)
                x_2_list.append(None)
            else:
                max_value = max(temp_y).item()
                temp_y_np = np.array([t.item() for t in temp_y])
                max_index = np.argmax(temp_y_np)
                # max_index = np.argmax(temp_y)
                c_x = temp_x[max_index]
                print("Early stop and feasible")
                print("obj= ",max_value, ' x = ', c_x)
                
                max_values.append(max_value)
                x_1_list.append(c_x[0].item())
                x_2_list.append(c_x[1].item())
            if len(iterations)==0:
                iterations.append(itera)
            else:
                iterations.append(itera-sum(iterations))
            temp_y.clear()
            temp_x.clear()

            df = pd.DataFrame({
            'obj': max_values,
            'x_1': x_1_list,
            'x_2': x_2_list,
            'iterations': iterations
            })
            if re_run_save == True:
                df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

run_multiple_calculations(num_runs=runs_number)

All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
Early stop and feasible
obj=  248.52925343003398  x =  tensor([0.5788, 0.0371], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
Early stop and feasible
obj=  239.24424190054623  x =  tensor([0.5126, 0.0913], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
Early stop and feasible
obj=  261.27405964828245  x =  tensor([0.5608, 0.0160], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
Early stop and feasible
obj=  215.67780561063267  x =  tensor([0.5387, 0.1364], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
Early stop and feasible
obj=  230.05095303304603  x =  tensor

In [44]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,5.000000,5.000000,5.000000,5.000000
mean,238.955263,0.536979,0.081065,106.600000
std,17.399714,0.034454,0.052967,2.073644
min,215.677806,0.494080,0.015988,105.000000
25%,230.050953,0.512607,0.037122,105.000000
50%,239.244242,0.538691,0.091289,106.000000
75%,248.529253,0.560753,0.124504,107.000000
max,261.274060,0.578763,0.136425,110.000000


### PSO

In [45]:
re_run = True
re_run_save = True

In [46]:
sol = 'PSO'
case_name = 'Case2'

x_1_list = []
x_2_list = []
obj_list = []
iterations=[]

runs_number = 100
itera = 0
from pyswarm import pso

def func(x):
    global itera
    obj, c1 = Case2(x)
    cons = [-c1]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

lb = [-5,0] 
ub = [10,15] 
if re_run == True or re_run_save == True:
    for num in range(runs_number):
        xopt, fopt = pso(lambda x:func(x)[0], lb, ub,f_ieqcons=lambda x:func(x)[1:2], swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=50, minstep=1e-8,minfunc=1e-8, debug=False)
        print( 'fopt =', fopt, 'xopt = ', xopt)
        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(fopt)
        x_1_list.append(xopt[0])
        x_2_list.append(xopt[1])

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'iterations': iterations
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


Stopping search: maximum iterations reached --> 50
fopt = -268.7885001160981 xopt =  [3.27270717 0.04901236]
Stopping search: maximum iterations reached --> 50
fopt = -268.78795696124433 xopt =  [3.26917401 0.0506208 ]
Stopping search: maximum iterations reached --> 50
fopt = -268.78848939169205 xopt =  [3.27352098 0.04864657]
Stopping search: maximum iterations reached --> 50
fopt = -268.7364663515497 xopt =  [3.23803231 0.06640501]
Stopping search: maximum iterations reached --> 50
fopt = -268.74230018065964 xopt =  [3.23749974 0.06645084]


In [47]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,5.000000,5.000000,5.000000,5.0
mean,-268.768743,3.258187,0.056227,5101.0
std,0.026881,0.018714,0.009342,0.0
min,-268.788500,3.237500,0.048647,5101.0
25%,-268.788489,3.238032,0.049012,5101.0
50%,-268.787957,3.269174,0.050621,5101.0
75%,-268.742300,3.272707,0.066405,5101.0
max,-268.736466,3.273521,0.066451,5101.0


### GA

In [48]:
re_run = True
re_run_save = True

In [51]:
sol = 'GA'
case_name = 'Case2'

x_1_list = []
x_2_list = []
obj_list = []
iterations=[]

runs_number = 100
itera = 0
from sko.GA import GA
from sko.operators import crossover

def func(x):
    global itera
    obj, c1= Case2(x)
    cons = [c1]
    itera=itera+1
    return obj,c1

lb = [-5,0] 
ub = [10,15] 
if re_run == True or re_run_save == True:
    for num in range(runs_number):
        ga = GA(func=lambda x:func(x)[0], n_dim=2, size_pop=100, max_iter=50, lb =[-5,0],ub=[10,15], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[1]])
        ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
        best_x, best_y = ga.run()
        print( 'obj =', best_y, 'x = ', best_x)

        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(best_y[0])
        x_1_list.append(best_x[0])
        x_2_list.append(best_x[1])

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'iterations': iterations
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

obj = [-266.30882944] x =  [3.35593104 0.09479363]
obj = [-266.15129746] x =  [3.01026219 0.25907524]
obj = [-255.5842421] x =  [2.66605402 0.79445606]
obj = [-267.08493198] x =  [3.05572457 0.20601573]
obj = [-256.26196683] x =  [2.68167415 0.76258192]


In [52]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,5.000000,5.000000,5.000000,5.0
mean,-262.278254,2.953929,0.423385,10001.0
std,5.817123,0.288156,0.329760,0.0
min,-267.084932,2.666054,0.094794,10001.0
25%,-266.308829,2.681674,0.206016,10001.0
50%,-266.151297,3.010262,0.259075,10001.0
75%,-256.261967,3.055725,0.762582,10001.0
max,-255.584242,3.355931,0.794456,10001.0
